In [1]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

import torch

from docx import Document

import os
import win32com.client

import openpyxl

In [2]:
# наші файли зберігаються у різних форматах, приведемо їх в один формат
files = os.listdir('Вікна')
for file in files:
    if file.split('.')[-1] == 'doc':
        word = win32com.client.Dispatch("Word.Application")
        doc = word.Documents.Open(f'C:\\Users\\lemes\\OneDrive\\Рабочий стол\\Домашки DataScience\\Вікна\\{file}')
        doc.SaveAs(f'C:\\Users\\lemes\\OneDrive\\Рабочий стол\\Домашки DataScience\\Вікна\\{file}x', 16) #16 значить формат docx


doc.Close()
word.Quit()

In [3]:
# функція з нашою моделью
model_name = 'timpal0l/mdeberta-v3-base-squad2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

def answer(question, text):
    # задаємо питання
    inputs = tokenizer(question, text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()    
    predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
    
    #отримаємо відповідь
    predicted_answer_text = tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)
    return predicted_answer_text
    
    

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


у мене в директорії для прикладу лежить 28 файлів з 1 до 28 також є 1 файл excel в якйи я вивів те що очікую від моделі

In [4]:
# Почнемо редагувати лист
wb = openpyxl.load_workbook('Проверочный лист.xlsx')
ws = wb.active

In [5]:
def create_answer_table(question, cell):
    for i in range(1,29): 
        doc_path = f'C:/Users/lemes/OneDrive/Рабочий стол/Домашки DataScience/Вікна/{i}.docx'
        doc = Document(doc_path)
        for paragraph in doc.paragraphs:
            if 'Дозвол' in paragraph.text or 'дозвол' in paragraph.text or 'надайте' in paragraph.text:
                ws[f'{cell}{i+1}'].value = answer(question=question, text=paragraph.text)
                break

In [6]:
create_answer_table('Де проводяться роботи?', 'D')
create_answer_table('В які дати проводяться роботи?', 'E')
wb.save('Проверочный лист.xlsx')

Приккріплю файл, із всіх відповідей що мені надала модель, тільки 1 мене не задовольняє, а так, як на мене - то дуже крутий результат.